In [1]:
# Import dependencies
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine

In [2]:
#Import functions
def model(df, position, var):
    df = df.dropna()
    df = df[df['pos'] == position]   

    # the model will use different independent variables depending on position
    if position == 'QB':
        X = df[var]
    elif position == 'RB':
        X = df[var]
    elif position == 'WR' or 'TE':
        X = df[var]
    else:
        print('Invalid position entered')
        return

    y = df['next_yr_ppg']
    reg = linear_model.LinearRegression()
    reg.fit(X, y)
    return reg

def testModelAccuracy(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return

    yTest = df['next_yr_ppg']
    results = model.score(XTest, yTest)
    return results

def testModelDifference(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return


    yPred = model.predict(XTest)
    predAndActual = {'Name': df['player'], 'Predicted PPG': yPred,
                     'Actual PPG': df['next_yr_ppg']}

    # creates df from dictionary above
    database = pd.DataFrame(predAndActual)

    # creates a difference column which depicts the difference between the
    # predicted PPG and actual PPG
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals=3)
    database['Difference'] = database['Predicted PPG'] - database['Actual PPG']
    database['Difference'] = database['Difference'].round(decimals=3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)

    return database, meanDiff, medianDiff, meanAbsDiff, medianAbsDiff

def testModel(model, test, train, position, var):
    accuracy = testModelAccuracy(model, train, position, var)
    differences = testModelDifference(model, test, position, var)
    meanDiff = differences[1]
    medDiff = differences[2]
    meanAbsDiff = differences[3]
    medAbsDiff = differences[4]

    print('The accuracy of the {0} model is {1}'.format(position, accuracy))
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medDiff, medAbsDiff))
    print('\n')
    return

def useModel(model, df1, df2, position, var):
    df1.dropna()
    df2.dropna()
    # checks where the Fant Pos is the position given and returns a data frame
    # with only the rows that include said position
    df1 = df1[df1['pos'] == position]
    df2 = df2[df2['pos'] == position]
    df2 = df2[['plid','player','ppr_g']]
    newdf = df1.merge(df2,how='inner',left_on='plid',right_on='plid')

    # the model will use difference parameters based on position
    if position == 'QB':
        X = df1[var]
    elif position == 'RB':
        X = df1[var]
    elif position == 'WR' or 'TE':
        X = df1[var]
    else:
        print('Invalid position entered')
        return
    yPred = model.predict(X)

    # creates new df with the name of player, their position, and their
    # predicted PPG
    databaseDict = {'Name': df1['player'], 'Pos': df1['pos'], 'Predicted PPG': yPred}
    database = pd.DataFrame(databaseDict)
    database = database.merge(df2,how='inner',left_on='Name',right_on='player')
    database = database.drop(columns=['player','plid'])
    database['Difference'] = database['Predicted PPG'] - database['ppr_g']
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals = 3)
    database['Predicted PPR'] = 17 * database['Predicted PPG']
    database['Predicted PPR'] = database['Predicted PPR'].round(decimals = 3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medianDiff, medianAbsDiff))


     # this adds a position rank column to the dataframe
#     posRank = []
#     posRankNum = 1
#     for index, row in newdf.iterrows():
#         posRank.append(posRankNum)
#         posRankNum += 1

#     database['PosRank'] = posRank


    database = database.reset_index(drop = True)

    return database

In [3]:
# Connect to Database
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'fballfinalproject.c6sg90iemyn2.us-east-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'FFForesight5!!' ## CHANGE THIS TO YOUR POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}').format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME)
# Create the connection
cnx = create_engine(postgres_str)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
# Read in dataframes and merge
# fantasy = pd.read_sql_query('''SELECT * FROM fantasy;''', cnx)
# teams = pd.read_sql_query('''SELECT * FROM teams;''', cnx)
# advrush = pd.read_sql_query('''SELECT * FROM advrush;''', cnx)
# df = pd.merge(fantasy,teams[['tmid','passrate']],on='tmid', how='inner')
# df = pd.merge(df,advrush,on='plid',how='inner')
# df = df.drop(columns=['player_y', 'tm_y', 'pos_y',
#                       'age_y','G_y', 'gs_y','tmid_y'])
# df = df.rename(columns={'player_x':'player','tm_x':'tm','pos_x':'pos','age_x':'age','G_x':'G','gs_x':'gs','tmid_x':'tmid'})
# df
df = pd.read_sql_query('''SELECT fantasy.*,teams.pasatt_g AS tm_passatt,teams.passrate,teams.rushatt_g AS tm_rushatt FROM fantasy INNER JOIN teams ON fantasy.tmid = teams.tmid;''',cnx)

In [5]:
df

,player,tm,pos,age,G,gs,plid,tmid,cmp_g,passatt_g,...,posrk,ovrank,yr,ppr_g,starter,next_yr_ppg,next_yr_starter,tm_passatt,passrate,tm_rushatt
0,AJBrown,TEN,WR,24,13,13,AJBrown2021,TEN2021,0.000,0.154,...,32,85,2021,13.915,0,NaN,NaN,31.471,90.0,32.412
1,AJBrown,TEN,WR,23,14,12,AJBrown2020,TEN2020,0.000,0.000,...,9,38,2020,17.679,0,13.915,0.0,30.312,105.9,32.562
2,AJBrown,TEN,WR,22,16,11,AJBrown2019,TEN2019,0.000,0.000,...,9,62,2019,13.569,0,17.679,0.0,28.000,108.3,27.812
3,AJDerby,MIA,TE,27,4,0,AJDerby2018,MIA2018,0.000,0.000,...,76,400,2018,3.450,0,NaN,NaN,28.438,93.0,23.188
4,AJDillon,GNB,RB,23,17,2,AJDillon2021,GNB2021,0.000,0.000,...,21,80,2021,10.918,0,NaN,NaN,34.882,107.4,26.235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5261,ZayJones,LVR,WR,25,16,2,ZayJones2020,LVR2020,0.062,0.062,...,130,321,2020,2.288,0,6.229,0.0,34.438,100.4,28.562
5262,ZayJones,BUF,WR,23,16,15,ZayJones2018,BUF2018,0.000,0.062,...,34,93,2018,10.325,0,3.260,0.0,31.188,62.6,29.250
5263,ZayJones,BUF,WR,22,15,10,ZayJones2017,BUF2017,0.000,0.000,...,90,218,2017,4.707,0,10.325,0.0,29.750,82.1,30.438
5264,ZurlonTipton,IND,RB,25,10,0,ZurlonTipton2015,IND2015,0.000,0.000,...,124,419,2015,1.270,0,NaN,NaN,38.688,77.5,24.750


In [6]:
# Check column choices
df.columns

Index(['player', 'tm', 'pos', 'age', 'G', 'gs', 'plid', 'tmid', 'cmp_g',
       'passatt_g', 'passyds_g', 'passtd_g', 'int_g', 'rushatt_g', 'rushyds_g',
       'rushyds_att', 'rushtd_g', 'tgt_g', 'rec_g', 'recyds_g', 'yds_rec',
       'rectd_g', 'fmb_g', 'fl_g', 'tottd_g', '2PM_G', '2PP_G', 'fpts',
       'posrk', 'ovrank', 'yr', 'ppr_g', 'starter', 'next_yr_ppg',
       'next_yr_starter', 'tm_passatt', 'passrate', 'tm_rushatt'],
      dtype='object')

In [7]:
# Manipulate df to add rush and target share
df['rushshare'] = df['rushatt_g'] / df['tm_rushatt']
df['targetshare'] = df['tgt_g'] / df['tm_passatt']
df

,player,tm,pos,age,G,gs,plid,tmid,cmp_g,passatt_g,...,yr,ppr_g,starter,next_yr_ppg,next_yr_starter,tm_passatt,passrate,tm_rushatt,rushshare,targetshare
0,AJBrown,TEN,WR,24,13,13,AJBrown2021,TEN2021,0.000,0.154,...,2021,13.915,0,NaN,NaN,31.471,90.0,32.412,0.004751,0.256649
1,AJBrown,TEN,WR,23,14,12,AJBrown2020,TEN2020,0.000,0.000,...,2020,17.679,0,13.915,0.0,30.312,105.9,32.562,0.000000,0.249769
2,AJBrown,TEN,WR,22,16,11,AJBrown2019,TEN2019,0.000,0.000,...,2019,13.569,0,17.679,0.0,28.000,108.3,27.812,0.006760,0.187500
3,AJDerby,MIA,TE,27,4,0,AJDerby2018,MIA2018,0.000,0.000,...,2018,3.450,0,NaN,NaN,28.438,93.0,23.188,0.000000,0.043955
4,AJDillon,GNB,RB,23,17,2,AJDillon2021,GNB2021,0.000,0.000,...,2021,10.918,0,NaN,NaN,34.882,107.4,26.235,0.419287,0.062382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5261,ZayJones,LVR,WR,25,16,2,ZayJones2020,LVR2020,0.062,0.062,...,2020,2.288,0,6.229,0.0,34.438,100.4,28.562,0.000000,0.036297
5262,ZayJones,BUF,WR,23,16,15,ZayJones2018,BUF2018,0.000,0.062,...,2018,10.325,0,3.260,0.0,31.188,62.6,29.250,0.002120,0.204406
5263,ZayJones,BUF,WR,22,15,10,ZayJones2017,BUF2017,0.000,0.000,...,2017,4.707,0,10.325,0.0,29.750,82.1,30.438,0.000000,0.165815
5264,ZurlonTipton,IND,RB,25,10,0,ZurlonTipton2015,IND2015,0.000,0.000,...,2015,1.270,0,NaN,NaN,38.688,77.5,24.750,0.020202,0.020678


In [8]:
# List out columns
df.columns

Index(['player', 'tm', 'pos', 'age', 'G', 'gs', 'plid', 'tmid', 'cmp_g',
       'passatt_g', 'passyds_g', 'passtd_g', 'int_g', 'rushatt_g', 'rushyds_g',
       'rushyds_att', 'rushtd_g', 'tgt_g', 'rec_g', 'recyds_g', 'yds_rec',
       'rectd_g', 'fmb_g', 'fl_g', 'tottd_g', '2PM_G', '2PP_G', 'fpts',
       'posrk', 'ovrank', 'yr', 'ppr_g', 'starter', 'next_yr_ppg',
       'next_yr_starter', 'tm_passatt', 'passrate', 'tm_rushatt', 'rushshare',
       'targetshare'],
      dtype='object')

In [9]:
# Select columns for each position (WR + TE joined in recvar)
qbvar = ['age','passatt_g','passyds_g','passtd_g','rushatt_g','rushyds_g',
         'rushyds_att','ppr_g','starter','next_yr_starter']
rbvar = ['age','rushatt_g','rushyds_g','rushyds_att','tgt_g','rec_g',
         'recyds_g','yds_rec','ppr_g','rushshare','targetshare']
recvar = ['age','tgt_g','rec_g','recyds_g','yds_rec','passrate','targetshare']

In [10]:
# Separate into training, testing, using, and actual data by Yr (Fantasy) or Year (other dataframes)
train = df[df['yr'] <= 2012]
test = df[(df['yr'] >= 2018) & (df['yr'] <= 2019)]
use = df[df['yr'] == 2019]
use = use[use['next_yr_starter'].notna()]
use = use.astype({"next_yr_starter": int})
actuals = df[df['yr'] == 2020]

In [11]:
# Name model and put in position
QBmodel = model(train, "QB", qbvar)
RBmodel = model(train, "RB", rbvar)
WRmodel = model(train, "WR", recvar)
TEmodel = model(train, "TE", recvar)
testModel(QBmodel, test, train, "QB", qbvar)
testModel(RBmodel, test, train, "RB", rbvar)
testModel(WRmodel, test, train, "WR", recvar)
testModel(TEmodel, test, train, "TE", recvar)

The accuracy of the QB model is 0.7700588664323179
The QB model has an average error of -1.18 PPG and an average absolute error of 4.097 PPG
The QB model has a median error of -0.477 PPG and a median absolute error of 3.409 PPG


The accuracy of the RB model is 0.5398155761161572
The RB model has an average error of 0.168 PPG and an average absolute error of 2.965 PPG
The RB model has a median error of 0.51 PPG and a median absolute error of 2.198 PPG


The accuracy of the WR model is 0.6151579335825005
The WR model has an average error of 0.79 PPG and an average absolute error of 3.194 PPG
The WR model has a median error of 1.194 PPG and a median absolute error of 2.834 PPG


The accuracy of the TE model is 0.5935201455571395
The TE model has an average error of 0.165 PPG and an average absolute error of 2.27 PPG
The TE model has a median error of 0.637 PPG and a median absolute error of 1.74 PPG




In [12]:
# Use model to make predictions and check predictions
QBStats = useModel(QBmodel, use, actuals, 'QB', qbvar)
RBStats = useModel(RBmodel, use, actuals,'RB', rbvar)
WRStats = useModel(WRmodel, use, actuals, 'WR', recvar)
TEStats = useModel(TEmodel, use, actuals, 'TE', recvar)

The QB model has an average error of -1.138 PPG and an average absolute error of 4.053 PPG
The QB model has a median error of -0.485 PPG and a median absolute error of 3.33 PPG
The RB model has an average error of -0.105 PPG and an average absolute error of 3.042 PPG
The RB model has a median error of 0.435 PPG and a median absolute error of 2.547 PPG
The WR model has an average error of 0.284 PPG and an average absolute error of 3.165 PPG
The WR model has a median error of 0.699 PPG and a median absolute error of 2.806 PPG
The TE model has an average error of -0.002 PPG and an average absolute error of 2.186 PPG
The TE model has a median error of 0.501 PPG and a median absolute error of 1.701 PPG


In [13]:
pd.set_option('display.max_rows', None)
QBStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,DakPrescott,QB,18.071,27.120,-9.049404,307.207,9.049404
1,LamarJackson,QB,17.833,22.187,-4.353558,303.161,4.353558
2,PatrickMahomes,QB,17.782,24.960,-7.178005,302.294,7.178005
3,DanielJones,QB,17.282,12.857,4.424539,293.794,4.424539
4,BakerMayfield,QB,17.216,15.538,1.678174,292.672,1.678174
5,KylerMurray,QB,17.081,23.669,-6.588009,290.377,6.588009
6,MattRyan,QB,17.058,17.650,-0.592213,289.986,0.592213
7,PhilipRivers,QB,16.924,15.000,1.923864,287.708,1.923864
8,DerekCarr,QB,16.654,17.006,-0.351694,283.118,0.351694
9,DeshaunWatson,QB,16.557,23.081,-6.524190,281.469,6.524190


In [14]:
QBStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
10,MatthewStafford,QB,16.420,16.288,0.131832,279.140,0.131832
51,JarrettStidham,QB,2.882,2.580,0.301683,48.994,0.301683
50,ChaseDaniel,QB,3.372,3.050,0.321720,57.324,0.321720
16,MitchellTrubisky,QB,15.706,15.370,0.335971,267.002,0.335971
8,DerekCarr,QB,16.654,17.006,-0.351694,283.118,0.351694
56,BrianHoyer,QB,1.622,2.000,-0.378345,27.574,0.378345
12,JaredGoff,QB,16.381,16.000,0.380567,278.477,0.380567
6,MattRyan,QB,17.058,17.650,-0.592213,289.986,0.592213
17,CarsonWentz,QB,15.644,16.533,-0.888586,265.948,0.888586
57,TimBoyle,QB,0.893,-0.112,1.004890,15.181,1.004890


In [15]:
RBStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,ChristianMcCaffrey,RB,22.818,30.133,-7.314681,387.906,7.314681
1,DalvinCook,RB,17.430,24.129,-6.698610,296.310,6.698610
2,AlvinKamara,RB,16.873,25.187,-8.314408,286.841,8.314408
3,LeonardFournette,RB,16.532,10.154,6.377600,281.044,6.377600
4,EzekielElliott,RB,15.781,14.913,0.867844,268.277,0.867844
5,AustinEkeler,RB,15.780,16.530,-0.749961,268.260,0.749961
6,SaquonBarkley,RB,15.077,7.700,7.377214,256.309,7.377214
7,DerrickHenry,RB,14.879,20.819,-5.939610,252.943,5.939610
8,NickChubb,RB,14.174,17.308,-3.133874,240.958,3.133874
9,ChrisCarson,RB,14.094,15.650,-1.555869,239.598,1.555869


In [16]:
RBStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
73,TravisHomer,RB,3.662,3.644,0.018000,62.254,0.018000
104,TonyBrooksJames,RB,0.377,0.400,-0.022933,6.409,0.022933
63,KyleJuszczyk,RB,4.984,5.100,-0.115821,84.728,0.115821
43,AdrianPeterson,RB,7.549,7.781,-0.231886,128.333,0.231886
100,BuddyHowell,RB,0.800,0.550,0.250088,13.600,0.250088
95,JakobJohnson,RB,1.348,1.094,0.254494,22.916,0.254494
47,AlexanderMattison,RB,7.006,6.685,0.321197,119.102,0.321197
102,SenorisePerry,RB,0.524,0.129,0.395290,8.908,0.395290
40,SonyMichel,RB,7.954,8.367,-0.413473,135.218,0.413473
61,BennySnellJr,RB,5.116,4.681,0.434939,86.972,0.434939


In [17]:
WRStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,MichaelThomas,WR,18.602,11.986,6.615561,316.234,6.615561
1,MichaelGallup,WR,18.295,10.831,7.463637,311.015,7.463637
2,MikeEvans,WR,17.505,15.538,1.967322,297.585,1.967322
3,JulioJones,WR,17.503,16.233,1.270252,297.551,1.270252
4,ChrisGodwin,WR,17.477,15.917,1.560353,297.109,1.560353
5,DavanteAdams,WR,15.720,25.600,-9.879767,267.240,9.879767
6,TyreekHill,WR,15.687,21.927,-6.239606,266.679,6.239606
7,AmariCooper,WR,15.497,14.800,0.697394,263.449,0.697394
8,DJMoore,WR,15.238,14.100,1.138491,259.046,1.138491
9,AJBrown,WR,14.833,17.679,-2.845977,252.161,2.845977


In [18]:
WRStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
89,ScottMiller,WR,6.399,6.406,-0.007185,108.783,0.007185
40,JohnBrown,WR,10.777,10.756,0.020650,183.209,0.020650
77,DavidMoore,WR,7.512,7.425,0.086515,127.704,0.086515
28,JarvisLandry,WR,12.363,12.533,-0.169658,210.171,0.169658
25,TylerBoyd,WR,12.624,12.840,-0.216170,214.608,0.216170
109,LaquonTreadwell,WR,4.823,4.580,0.243325,81.991,0.243325
34,MarquiseBrown,WR,11.706,11.438,0.268179,199.002,0.268179
139,AndreRoberts,WR,0.564,0.207,0.356689,9.588,0.356689
138,RobertFoster,WR,1.067,1.425,-0.358134,18.139,0.358134
45,RandallCobb,WR,10.461,10.010,0.451479,177.837,0.451479


In [19]:
TEStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,TravisKelce,TE,15.226,20.853,-5.626852,258.842,5.626852
1,GeorgeKittle,TE,15.179,15.638,-0.459157,258.043,0.459157
2,DarrenWaller,TE,14.124,17.412,-3.288054,240.108,3.288054
3,MarkAndrews,TE,13.274,12.150,1.124238,225.658,1.124238
4,ZachErtz,TE,13.256,7.045,6.210909,225.352,6.210909
5,EvanEngram,TE,12.814,8.812,4.001849,217.838,4.001849
6,AustinHooper,TE,11.107,8.731,2.376188,188.819,2.376188
7,HunterHenry,TE,10.880,10.379,0.500733,184.960,0.500733
8,JaredCook,TE,9.688,8.493,1.194883,164.696,1.194883
9,TylerHigbee,TE,9.418,8.547,0.870913,160.106,0.870913


In [20]:
TEStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
54,MaxxWilliams,TE,2.648,2.689,-0.041151,45.016,0.041151
78,DeonYelder,TE,0.826,0.757,0.068780,14.042,0.068780
31,KyleRudolph,TE,5.373,5.450,-0.077359,91.341,0.077359
40,DavidNjoku,TE,4.107,4.023,0.084206,69.819,0.084206
79,ChrisManhertz,TE,0.813,0.700,0.113267,13.821,0.113267
67,DarrellDaniels,TE,1.780,1.933,-0.153085,30.260,0.153085
57,MarcedesLewis,TE,2.422,2.580,-0.157670,41.174,0.157670
27,JordanAkins,TE,6.131,6.438,-0.306932,104.227,0.306932
33,DarrenFells,TE,5.123,4.762,0.360912,87.091,0.360912
63,VirgilGreen,TE,1.960,2.333,-0.372958,33.320,0.372958
